In [1]:

import pickle
import os
my_bucket = os.getenv("WORKSPACE_BUCKET")

# preparation

## load rsid 

In [ ]:
import pickle
import pandas as pd

personids = pickle.load( open('personids.pkl', 'rb'))
patient_cols = [str(int(i)) for i in personids if pd.notnull(i)]
print(len(patient_cols), patient_cols[:3]) 
select_snps = pickle.load(open('select_snps.pkl', 'rb'))
gwas_snps = select_snps

## fetch locations of snps


In [ ]:

chr_list = gwas_snps['CHR_ID'].values.tolist()
chr_pos_list = gwas_snps['CHR_POS'].values.tolist()
rsid_list = gwas_snps['SNPS'].values.tolist()

failload_rowcount = 0
select_snps = []
for i in  range(len(chr_list)):
    chrID =  str( chr_list[i] )
    chr_pos = str(chr_pos_list[i])
    rsid = str(rsid_list[i] )

    try:
        if ';' in chrID:
            split_chrID = [c.strip() for c in chrID.split(';')]
            split_chr_pos = [cp.strip() for cp in chr_pos.split(';')]
            split_rsid = [r.strip() for r in rsid.split(';')]

            tuples = [(j,k, l) for j,k,l in zip(split_chrID, split_chr_pos, split_rsid)]
            select_snps.extend(tuples)
        elif 'x' in chrID and chrID != 'X':
            split_chrID = [c.strip() for c in chrID.split('x')]
            split_chr_pos = [cp.strip() for cp in chr_pos.split('x')]
            split_rsid = [r.strip() for r in rsid.split('x')]

            tuples = [(j,k, l) for j,k,l in zip(split_chrID, split_chr_pos, split_rsid)]
            select_snps.extend(tuples)
        elif ' - ' in chrID:
            split_chrID = [c.strip() for c in chrID.split('-')]
            split_chr_pos = [cp.strip() for cp in chr_pos.split('-')]
            split_rsid = [r.strip() for r in rsid.split('-')]

            tuples = [(j,k, l) for j,k,l in zip(split_chrID, split_chr_pos, split_rsid)]
            select_snps.extend(tuples)
        else:
            select_snps.append((chrID, chr_pos, rsid))
    except:
        print(i,type( chrID), type( chr_pos),type(  rsid))

        failload_rowcount += 1
        flag = 55
        
print(failload_rowcount) # 0



In [ ]:
gwas_location = pd.DataFrame(select_snps)
gwas_location.columns = ['chrom', 'pos', 'rsid']
print(gwas_location.shape)  

gwas_location = gwas_location[gwas_location.chrom!='nan']
print(gwas_location.shape)  

gwas_location_unique = gwas_location.drop_duplicates()
print('gwas_location_unique', gwas_location_unique.shape)


##  convert to locus

In [ ]:

get_snps_location = gwas_location_unique[['chrom', 'pos', 'rsid']]
get_snps_location

In [ ]:

import hail as hl
get_snps_location['chrom'] = 'chr' + get_snps_location['chrom']
ht_loci = hl.Table.from_pandas(get_snps_location)

ht_loci = ht_loci.annotate(locus=hl.locus(ht_loci.chrom, ht_loci.pos, reference_genome='GRCh38'))
ht_loci = ht_loci.key_by('locus')
ht_loci.show(n=5)
print("Number of rows:", ht_loci.count())


# query gene data

load srWGS acaf hail-split and select cols and rows 

In [ ]:
import os
import pandas as pd
gen_path = os.getenv("WGS_ACAF_THRESHOLD_SPLIT_HAIL_PATH")
gen_path

In [125]:
import hail as hl
mt = hl.read_matrix_table(gen_path)

In [ ]:
mt.describe(widget=True)

In [ ]:
print('filter columns for personids', len(patient_cols))
patient_mt = mt.filter_cols(hl.literal(patient_cols).contains(mt.s))


In [ ]:

to_locus_set = [locus for locus in ht_loci.locus.collect()]
print('to locus ', len(to_locus_set))


In [ ]:
from tqdm import tqdm 

batch_size = 100
batches  = [to_locus_set[i:i + batch_size] for i in range(0, len(to_locus_set), batch_size)]
print('Total batches:', len(batches))

batches = batches
for b, batch_locus_set in tqdm(enumerate(batches)):

    locus_set = hl.literal(batch_locus_set)
    
    filtered_mt = patient_mt.filter_rows(locus_set.contains(patient_mt.locus))
    print(f'After  filter: {filtered_mt.count_rows()} rows')
    entries_df = filtered_mt.entries()

    entries_df = entries_df.key_by()

    selected_entries = entries_df.select(
        entries_df.locus,
        entries_df.alleles,
        entries_df.s,
        entries_df.GT,
    )
    
    selected_entries.export(f'genetic_data/variant_{b}.tsv.bgz')
